In [1]:
import librosa
import numpy as np
import os
import pandas as pd
import pickle
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model

torch.cuda.is_available()

True

In [2]:
emotion_categories = ['NEU', 'HAP', 'SAD', 'ANG', 'FEA', 'DIS']

In [3]:
data_dir = "../data/CREMA-D/"

audio_labels = []
audio_arrays = []
file_names = []

for file in os.listdir(data_dir):
    for emotion in emotion_categories:
        if emotion in file:
            audio_labels.append(emotion)
            audio, sr = librosa.load(os.path.join(data_dir, file), sr=16000)
            audio_arrays.append(audio)
            file_names.append(file)

In [4]:
df = pd.DataFrame()
df["file_name"] = file_names
df["audio_array"] = audio_arrays
df["label"] = audio_labels

In [5]:
df.head()

,file_name,audio_array,label
0,1001_DFA_ANG_XX.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",ANG
1,1001_DFA_DIS_XX.wav,"[0.00091552734, 0.0013122559, 0.0015869141, 0....",DIS
2,1001_DFA_FEA_XX.wav,"[0.008026123, 0.008453369, 0.008361816, 0.0074...",FEA
3,1001_DFA_HAP_XX.wav,"[-0.004333496, -0.0032653809, -0.0030212402, -...",HAP
4,1001_DFA_NEU_XX.wav,"[0.0024108887, 0.001373291, 0.0013122559, 0.00...",NEU


In [6]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

def extract_embeddings(audio_samples, batch):
    
    print(batch)
    embeddings_list = []
    
    for audio_array in audio_samples:
        # Converting the audio array into a PyTorch tensor
        input_values = processor(audio_array, return_tensors="pt").input_values

        # Obtaining the sequence of hidden states at the output of the last layer of the model
        hidden_states = model(input_values).last_hidden_state

        # Applying a pooling method to extract vector embeddings from the output of the final hidden layer 
        embeddings = torch.mean(hidden_states, dim=1)
        embeddings_list.append(embeddings)
        
    return embeddings_list

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# embeddings_array = []

batch_size = 128
batch_number = 1
for i in range(0, len(audio_arrays), batch_size):
    
    embeddings_list = extract_embeddings(audio_arrays[i : i+batch_size], batch_number)
    batch_number += 1